<a href="https://colab.research.google.com/github/VictorSepulveda1994/Minor-Machine-Learning/blob/main/Proyecto_Final/Etapa02-Scrub/Scrub-Human_Activity_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto Final - Reconocimiento de Actividad Humana (HAR)

In [43]:
# Librerías más utilizadas
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder

## **Entrega 01: OBTAIN**
En esta etapa del proyecto será necesario obtener los datos de las fuentes disponibles. En este caso se utilizará un conjunto de datos públicos de reconocimiento de actividad humana, disponible en los links entregados en el enunciado del proyecto.

### Procedimiento
 
 A continuación, se detallan los pasos realizados en ésta etapa.

#### 1. Comprobación de los links



*   El primer link en el enunciado de [cis.fordham.edu](https://www.cis.fordham.edu/wisdm/dataset.php) funciona correctamente.


*   El segundo link [archive.ics.uci.edu](https://archive.ics.uci.edu/ml/datasets/WISDM+Smartphone+and+Smartwatch+Activity+and+Biometrics+Dataset) no funciona correctamente y aparece un mensaje mencionando que los datos no aparecen o no existen.

Por lo tanto, se utilizará la fuente del primer link.

#### 2. Descarga de los datos



Dado que los archivos están comprimidos en formato "tar.gz", y la plataforma de github solo soporta archivo de hasta 25Mb, para este caso se emplearán dos métodos para utilizarlos en Colab:

##### 2.1. Método Automático (*Recomendado*)

In [44]:
#Se instala el módulo "wget" con el siguiente comando
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [45]:
#Se importa el módulo "wget"
import wget

In [46]:
#Se asigna el link del botón de descarga de la página cis.fordham.edu
url = "https://www.cis.fordham.edu/wisdm/includes/datasets/latest/WISDM_ar_latest.tar.gz"

In [47]:
#Se descarga el "archivo.tar.gz" con todo su contenido
wget.download(url, '/content/WISDM_ar_v1.1_raw.tar.gz')

'/content/WISDM_ar_v1.1_raw.tar.gz'

In [48]:
#Se descomprime el archivo "WISDM_ar_v1.1_raw.tar.gz"
import tarfile 
file = tarfile.open('WISDM_ar_v1.1_raw.tar.gz') 
print(file.getnames()) 
file.extractall('./')
file.close() 

['WISDM_ar_v1.1', 'WISDM_ar_v1.1/readme.txt', 'WISDM_ar_v1.1/WISDM_ar_v1.1_raw.txt', 'WISDM_ar_v1.1/WISDM_ar_v1.1_raw_about.txt', 'WISDM_ar_v1.1/WISDM_ar_v1.1_transformed.arff', 'WISDM_ar_v1.1/WISDM_ar_v1.1_trans_about.txt']




> **NOTA:** El archivo de texto que se utilizará "WISDM_ar_v1.1_raw.txt", estará dentro de la carpeta "WISDM_ar_v1.1".

##### 2.2. Método Manual (*Omitir si usó el Método Automático*)

* Ir a [cis.fordham.edu](https://www.cis.fordham.edu/wisdm/dataset.php)


* En la sección "Activity Prediction", hacer click en el botón "Download Latest Version" para descargar el archivo comprimido al computador.

* Descomprimir el archivo.tar.gz

* Subir el archivo "WISDM_ar_v1.1_raw.txt" al almacenamiento de sesión de Colab con el siguiente código.

In [49]:
#from google.colab import files #Descomentar si es necesario.
#files.upload()                 #Descomentar si es necesario.

* Esperar a que se suba el ".txt", ya que, éste contiene una gran cantidad de datos (49Mb aproximadamente).

#### 3. Vista de los datos

Ahora que los datos ya están descargados y subidos a Colab, es posible utilizarlos. Para ello, se realizará una vista general de los datos antes de continuar con la siguiente etapa.

In [50]:
# Cargar el archivo de texto. La ruta dependerá de como el archivo se descargó (método automático o manual).
df = pd.read_table("./WISDM_ar_v1.1/WISDM_ar_v1.1_raw.txt", header=None)

In [51]:
# Dataframe con los datos
df.head()

,0
0,"33,Jogging,49105962326000,-0.6946377,12.680544..."
1,"33,Jogging,49106062271000,5.012288,11.264028,0..."
2,"33,Jogging,49106112167000,4.903325,10.882658,-..."
3,"33,Jogging,49106222305000,-0.61291564,18.49643..."
4,"33,Jogging,49106332290000,-1.1849703,12.108489..."


## **Entrega 02: SCRUB**
En esta etapa del proyecto será necesario limpiar y filtrar los datos.

### Procedimiento
 A continuación, se detallan los pasos realizados en ésta etapa.

#### 1. Eliminar filas con datos faltantes

Uno de los pasos iniciales en la limpieza de datos consiste en eliminar aquellas filas que contengan datos faltantes. Estas líneas generalmente no aportan información y pueden complicar la etapa de entrenamiento y validación de muchos modelos predictivos.

In [52]:
#Se vuelve a procesar el archivo "WISDM_ar_v1.1_raw.txt"
file = open("./WISDM_ar_v1.1/WISDM_ar_v1.1_raw.txt")
lines = file.readlines()

processedList = []

# Se recorre el archivo, eliminando filas con datos faltantes y separando las "columnas"
for i, line in enumerate(lines):
    try:
        line = line.split(',')
        last = line[5].split(';')[0]
        last = last.strip()
        if last == '':
            break;
        temp = [line[0], line[1], line[2], line[3], line[4], last]
        processedList.append(temp)
    except:
        print('Error at line number: ', i)

Error at line number:  281873
Error at line number:  281874
Error at line number:  281875


#### 2. Agregar al DataFrame los nombres de las columnas

En ocasiones al crear los data frames el sistema asigna nombres genéricos y correlativos a las columnas. Es recomendable renombrar las columnas por nombres significativos para favorecer el análisis posterior de los datos.

In [53]:
# Se crea un array con los nombres de las columnas
columns = ['user', 'activity', 'time', 'x', 'y', 'z']
# Los nombres de las columnas y la lista procesada son insertadas en un nuevo dataframe
data = pd.DataFrame(data = processedList, columns = columns)

#### 3. Visualizar las 5 primeras filas del DataFrame (.head())

Siempre es buena medida visualizar algunas líneas del dataframe para tener una idea concreta del contenido.

In [54]:
data.head(5)

,user,activity,time,x,y,z
0,33,Jogging,49105962326000,-0.6946377,12.680544,0.50395286
1,33,Jogging,49106062271000,5.012288,11.264028,0.95342433
2,33,Jogging,49106112167000,4.903325,10.882658,-0.08172209
3,33,Jogging,49106222305000,-0.61291564,18.496431,3.0237172
4,33,Jogging,49106332290000,-1.1849703,12.108489,7.205164


#### 4. Buscar datos nulos y eliminarlos si es que existen

In [55]:
data.isnull().sum()

user        0
activity    0
time        0
x           0
y           0
z           0
dtype: int64

> **NOTA:** No hay datos nulos.



#### 5. Visualizar los tipo de datos a los que pertenece cada columna (.info())

Es necesario tener un detalle de cuantas variables están considerando y el tipo de datos que pertenecen para verificar si es posible utilizarla directamente para el modelo predictivo o si es necesario realizar una transformación del tipo de dato.